In [2]:
# 📌 Step 1: Install Dependencies
!pip install xgboost scikit-learn pandas numpy tensorflow

In [3]:
# 📌 Step 2: Import Libraries
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [4]:
# 📌 Step 3: Load and Explore Data
df = pd.read_csv("/content/customer_churn.csv")
df.head()

,CustomerID,Tenure,MonthlySpend,ConsultingHours,SatisfactionScore,ServiceUsage,ContractType,Churn
0,1,39,4263.30,42,2,0.647185,Fixed,0
1,2,52,7868.22,72,1,0.600870,Fixed,0
2,3,29,6429.68,24,3,0.764955,Fixed,0
3,4,15,8069.94,80,3,0.627432,Flexible,0
4,5,43,9041.20,49,3,0.632588,Flexible,1


In [5]:
# Encode ContractType
df['ContractType'] = df['ContractType'].apply(lambda x: 1 if x == "Fixed" else 0)

In [6]:
# Separate features and label
X = df.drop(['CustomerID', 'Churn'], axis=1)
y = df['Churn']

In [7]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Save scaler for future use
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [10]:
# Train XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:11:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [11]:
# Evaluate
y_pred_xgb = xgb_model.predict(X_test)
print("XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))


XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84       163
           1       0.14      0.08      0.10        37

    accuracy                           0.73       200
   macro avg       0.47      0.48      0.47       200
weighted avg       0.68      0.73      0.71       200



In [12]:
# Save XGBoost model
with open("xgboost_churn_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)

In [13]:
# Reshape input to be 3D [samples, timesteps, features]
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))


In [14]:
# Train/Test split
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(
    X_lstm, y, test_size=0.2, random_state=42
)

In [15]:
# Build LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=(X_lstm.shape[1], X_lstm.shape[2]), return_sequences=False))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(32, activation='relu'))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:

# Train LSTM
history = model_lstm.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32,
                         validation_data=(X_test_lstm, y_test_lstm), verbose=1)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.7352 - loss: 0.6674 - val_accuracy: 0.8150 - val_loss: 0.6225
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8138 - loss: 0.6077 - val_accuracy: 0.8150 - val_loss: 0.5676
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8122 - loss: 0.5512 - val_accuracy: 0.8150 - val_loss: 0.5240
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8070 - loss: 0.5164 - val_accuracy: 0.8150 - val_loss: 0.5005
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7927 - loss: 0.5120 - val_accuracy: 0.8150 - val_loss: 0.4927
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8125 - loss: 0.4748 - val_accuracy: 0.8150 - val_loss: 0.4910
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7923 - loss: 0.5010 - val_accuracy: 0.8150 - val_loss: 0.4881
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8168 - loss: 0.4659 - val_accuracy: 0.8150 - v

In [17]:
# Evaluate
lstm_preds = (model_lstm.predict(X_test_lstm) > 0.5).astype("int32")
print("LSTM Classification Report:\n", classification_report(y_test_lstm, lstm_preds))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step
LSTM Classification Report:
               precision    recall  f1-score   support

           0       0.81      1.00      0.90       163
           1       0.00      0.00      0.00        37

    accuracy                           0.81       200
   macro avg       0.41      0.50      0.45       200
weighted avg       0.66      0.81      0.73       200



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
# Save LSTM model
model_lstm.save("lstm_churn_model.h5")